# Input Perterbation

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




import pandas as pd

with open('/home/23m1526/ua.base', 'r') as file:
    # Read the lines of the file
    lines = file.readlines()

# Process the lines to create a list of tuples
data = []
for line in lines:
    parts = line.strip().split('\t')
    user_id = int(parts[0])
    item_id = int(parts[1])
    rating = int(parts[2])
    # Skip the timestamp (parts[3])
    data.append((user_id, item_id, rating))

# Create a DataFrame from the list of tuples
df = pd.DataFrame(data, columns=['userId', 'movieId', 'rating'])

# Print the first few rows of the DataFrame

df['movieId'].max()
df['movieId'].min()

temp = 1
userItem  = np.zeros((df['userId'].max()+1,df['movieId'].max()+1))
print(userItem )
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(944, 1683)


In [3]:
for i in range (len(df['userId'])):
  userItem[df.userId[i]][df.movieId[i]] = df.rating[i]

print(userItem)
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 5. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]
(944, 1683)


In [4]:
R = userItem[1:]
print(R)
R = R[:, 1:]
print(R)

ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

[[0. 5. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [5]:
A = R>0.5; A[A == True] = 1; A[A == False]   = 0; A = A.astype(np.float64, copy=False)


In [13]:
def input_perterbation_sgd(R,A, d, gamma, lambd, epsilon):
    k = 50
    num_users, num_items = R.shape
    P = np.random.rand(num_users, d)
    Q = np.random.rand(num_items, d)
    R1 = R + np.random.laplace(scale=(4) / epsilon, size=R.shape)
    R0 = np.zeros((R.shape))
    B =1
    for i in range(R.shape[0]):
        for j in range (R.shape[1]): 
            if A[i,j] != 0: R0[i,j ] = R1[i,j]

    
    R0 = np.clip(R0, -B, B)
    # Assuming R is a numpy array, find non-zero (i.e., rated) entries
    rated_indices = np.argwhere(R > 0)

    for epoch in range(k):
        np.random.shuffle(rated_indices)  # Shuffle the observed ratings
        for i, j in rated_indices:
            e_ui = R[i, j] - np.dot(P[i], Q[j]) 
       

            # Update rule remains the same, but only applied to observed ratings
            P[i] += gamma * (e_ui * Q[j] - lambd * P[i])
            Q[j] += gamma * (e_ui * P[i] - lambd * Q[j])

    return P, Q , P.dot(Q.T)


In [14]:
p,q, r =  input_perterbation_sgd(R,A, 3,0.01,0.6,  epsilon = 2  )

In [15]:
print(r)

r

[[3.30776698 2.71319155 2.59754634 ... 1.6878042  2.74767485 2.47883917]
 [3.27109245 2.68310819 2.56874653 ... 1.66995303 2.71732374 2.451556  ]
 [2.83546952 2.3257887  2.2266582  ... 1.44802265 2.35542265 2.12485221]
 ...
 [3.45142828 2.83103058 2.7103612  ... 1.76028046 2.86693978 2.58645172]
 [3.72334858 3.05407081 2.92389746 ... 1.90064016 3.09297545 2.79040042]
 [3.23513039 2.65361116 2.54050565 ... 1.65096521 2.68739133 2.42455002]]


array([[3.30776698, 2.71319155, 2.59754634, ..., 1.6878042 , 2.74767485,
        2.47883917],
       [3.27109245, 2.68310819, 2.56874653, ..., 1.66995303, 2.71732374,
        2.451556  ],
       [2.83546952, 2.3257887 , 2.2266582 , ..., 1.44802265, 2.35542265,
        2.12485221],
       ...,
       [3.45142828, 2.83103058, 2.7103612 , ..., 1.76028046, 2.86693978,
        2.58645172],
       [3.72334858, 3.05407081, 2.92389746, ..., 1.90064016, 3.09297545,
        2.79040042],
       [3.23513039, 2.65361116, 2.54050565, ..., 1.65096521, 2.68739133,
        2.42455002]])

In [ ]:
import math
def RMSE(actual, predicted):
    err = 0
    
    count = np.count_nonzero(actual)
    for i in range(actual.shape[0]):
        for j in range(actual.shape[1]):
            if actual[i][j] > 0:
                
                err += (actual[i][j] - predicted[i][j])**2
    
    if count == 0: "error is not calculated therefore count = 0 "
    else : 
        mse = err/count
    rmse = math.sqrt(mse)
    
    return  rmse
    
l =RMSE(R,r) 
print(l)

# input_perterbation_sgd(R,A, d, gamma, lambd, epsilon):

In [ ]:
ISGD_Rmse = []
ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
for _ in (ep):

    p,q , r = input_perterbation_sgd(R,A, 3,0.01,0.06,  epsilon = _  )

    temp = RMSE(R,r)
    ISGD_Rmse.append(temp)

In [ ]:
print(ISGD_Rmse)

In [39]:
import csv



csv_file_path = 'output2.csv'


In [42]:
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    writer.writerow(['ISGD_Rmse'] + ISGD_Rmse)